# Information Retrieval and Web Analytics: Indexing + Modeling (TF-IDF) 

In [1]:
# if you do not have nltk the following command should work "python -m pip install nltk" 
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to /home/waze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import re
import csv
import sys

data_path = './data/one2.json'

#### Load data into memory

As mentioned above the dataset is stored in a tsv file ```parsed_input_500.tsv```and it contains 500 Wikipedia article (one article per line). For each article we have id, title and body separated by "|".

In [3]:
docs_path = data_path
with open(docs_path) as fp:
    lines = fp.readlines()
tweets = [l.strip().replace(' +', ' ') for l in lines]

In [4]:
print("Total numer of tweets in the corpus: {}" .format(len(tweets)))

Total numer of tweets in the corpus: 1001


In [5]:
def cleanTweet(tweetText):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    ## START CODE
    line= tweetText.lower() ## Transform in lowercase
    line=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',line)
    line= line.split() ## Tokenize the text to get a list of terms
    line= [word for word in line if word not in stops]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemming.stem(word) for word in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line
    

In [6]:
def getRelevantInfo(tweet):
    dictRelevantInfo ={}
    data = json.loads(tweet)
    hashtags = []
    urlsList = []
    text = ''
    date = data['created_at'] ## ??? RT o no RT
    try:
        isRt=True
        isRetweet=data["retweeted_status"]
        idTweet=isRetweet["id_str"]
        text = isRetweet['text']
        username = isRetweet['user']['screen_name']
        urls = isRetweet['entities']['urls']
        rt_count = isRetweet['retweet_count']
        likes = isRetweet['favorite_count']
        
        for h in isRetweet['entities']['hashtags']:
            hashtags.append(h['text'])
        for url in urls:
            urlsList.append(url['url'])
            
    except:
        isRt=False
        idTweet=data["id_str"]
        text = data['text']
        username = data['user']['screen_name']
        urls = data['entities']['urls']
        rt_count=data['retweet_count']
        likes = data['favorite_count']
        
        for h in data['entities']['hashtags']:
            hashtags.append(h['text'])
            
        for url in urls:
            urlsList.append(url['url'])        
            
    dictRelevantInfo['tweetID'] = idTweet
    dictRelevantInfo['text'] = text
    dictRelevantInfo['tokens'] = cleanTweet(text)
    dictRelevantInfo['username'] = username
    dictRelevantInfo['date'] = date
    dictRelevantInfo['hashtags'] = hashtags
    dictRelevantInfo['likes'] = likes
    dictRelevantInfo['rt_count'] = rt_count
    dictRelevantInfo['urlsList'] = urlsList
    dictRelevantInfo['isRetweeted'] = isRt
    return dictRelevantInfo

In [7]:
cleanTweets = {}
for t in tweets:
    currentTweet=getRelevantInfo(t)
    tweetID=currentTweet['tweetID']
    isRtCurrent=currentTweet['isRetweeted']
    # Orignial tweet found, overwrite if retweet exist.
    if isRtCurrent == False:
        cleanTweets[tweetID] = currentTweet
    else:
        if tweetID in cleanTweets:
            continue
        else:
            cleanTweets[tweetID] = currentTweet

In [8]:
print("Length of cleaned tweets: ",len(cleanTweets))

Length of cleaned tweets:  709


In [9]:
def create_index(lines):
    """
    Impleent the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    """
    index=defaultdict(list) 
    tweetIndex = {} # dictionary to map tweets to page ids
    for line in lines.values(): # Remember, lines contain all tweets, each line is a tweet
        tweetID = line['tweetID']
        terms = line['tokens'] #page_title + page_text
        tweetIndex[tweetID]=line['text']  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
        ## START CODE
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweetID, array('I',[position])] #'I' indicates unsigned int (int in python)
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
        ## END CODE                    
                    
    return index, tweetIndex

In [10]:
start_time = time.time()

# index is the dict of , index by twee ?¿?¿
# tweetIndex is the dict of tweets, indexed by tweetID
index, tweetIndex = create_index(cleanTweets)

print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 0.01 seconds


Notice that if you look in the index for ```researcher```you will not find any result, while if you look for ```research``` you will get some results. That happens because we are storing in the index stemmed terms.

In [11]:
#print("Index results for the term 'biden': {}\n".format(index['biden']))
print("Length of Index results for the term 'biden': {}\n".format(len(index['obama'])))
print("First 10 Index results for the term 'biden': \n{}".format(index['biden'][:10]))

Length of Index results for the term 'biden': 8

First 10 Index results for the term 'biden': 
[['1331952640574459905', array('I', [1])], ['1333092424936284163', array('I', [3])], ['1333092425611554823', array('I', [8])], ['1333088877343608832', array('I', [6])], ['1333026911799439361', array('I', [7])], ['1332712274000293889', array('I', [3])], ['1333092426953658368', array('I', [7])], ['1333023981646057473', array('I', [6])], ['1333087495509012480', array('I', [0])], ['1333088498975436804', array('I', [2])]]


In [12]:
def getResultsFromIndex(word, index):
    stemming = PorterStemmer()
    newWord=word.lower() ## Transform in lowercase
    newWord=re.sub('[:\[\]&%$\"\'!./,;:?=¿^\-#_*+)<>(¡@]','',word)
    newWord= stemming.stem(word) ## perform stemming
    print("Length of Index results for the term '{}': {}".format(newWord, len(index[newWord])))
    print("First 10 Index results for the term '{}': \n{}\n\n".format(newWord, index[newWord][:10]))

In [13]:
getResultsFromIndex("trump", index)
getResultsFromIndex("biden", index)
getResultsFromIndex("joe", index)

Length of Index results for the term 'trump': 326
First 10 Index results for the term 'trump': 
[['1332994175281868801', array('I', [6])], ['1333068258862370819', array('I', [0])], ['1333068652665516036', array('I', [3])], ['1332927520270913536', array('I', [8])], ['1333057200802107393', array('I', [9])], ['1333091717579501569', array('I', [2])], ['1333071374081011715', array('I', [5])], ['1333023981646057473', array('I', [3])], ['1333070773758685194', array('I', [0])], ['1332844785334358016', array('I', [1])]]


Length of Index results for the term 'biden': 155
First 10 Index results for the term 'biden': 
[['1331952640574459905', array('I', [1])], ['1333092424936284163', array('I', [3])], ['1333092425611554823', array('I', [8])], ['1333088877343608832', array('I', [6])], ['1333026911799439361', array('I', [7])], ['1332712274000293889', array('I', [3])], ['1333092426953658368', array('I', [7])], ['1333023981646057473', array('I', [6])], ['1333087495509012480', array('I', [0])], ['1333

### Querying the Index

Even if before we mentioned that in case of phrase queries we need to take into account the position of the terms in the document and we have implemented an index that would allow us to also work with this type of queries, here you are going to implement a search function that will query the index without take into account the trems' positions.


We will use english Free Text Queries, that means that the query we will query the index using  a sequence of english words as query, and the output will be the list of documents that contain any of the query terms. 

For instance if we write the query **"computer science"** the output will be the union of all documents containing the term "computer" with all documents containing the term "science".

In [14]:
def search(query, index):
    '''
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=cleanTweet(query)
    tweets=set()
    for term in query:
    ## START DODE
        try:
            # store in termTweets the ids of the tweets that contain "term"                        
            termTweets=[posting[0] for posting in index[term]]
            # tweets = tweets Union termTweets
            tweets = tweets.union(termTweets)
        except:
            #term is not in index
            pass
    tweets=list(tweets)
    return tweets

In [15]:
print("Insert your query:\n")
query = input()
docs = search(query, index)    
top = 10

print("\n======================\nSample of {} results out of {} for the seached query:\n".format(top, len(docs)))
for d_id in docs[:top] :
    print("tweet_id= {}\ntweet_text: {}\n\n\n".format(d_id, tweetIndex[d_id]))

Insert your query:

biden

Sample of 10 results out of 155 for the seached query:

tweet_id= 1333051565352038400
tweet_text: I'll never call Biden my President, 
I'll never bow to Socialism &amp; I'll never accept the Radical agenda the left wa… https://t.co/S722x9pzMI



tweet_id= 1291460348852264961
tweet_text: 🔥Betrayal: SEAL Team 6 May 3 2011, at a national event in DC, VP Biden did the unthinkable - He publicly revealed t… https://t.co/PIfzZBsOTK



tweet_id= 1333068370292445188
tweet_text: The Pennsylvania supreme Court changed the election laws to benefit Joe Biden and the Democrats! Subscribe for free… https://t.co/DIXwV4awF3



tweet_id= 1333090140168839168
tweet_text: Biden won again



tweet_id= 1333023981646057473
tweet_text: HEROES AT EPA HOLDING OFF TRUMP'S REVENGE ORDERS
But now, with the Biden administration on the horizon, career E.P.… https://t.co/3BSO8qcZXN



tweet_id= 1333092459933540354
tweet_text: @lilbratley Biden is being tone deaf. With most people having 60k 

### Ranking tf-idf

When searching in a search engine, we are interested in obtain the results sorted by relevance or by some other criteria. Notice that **the above results are not ranked**.

Here you are going to implement **tf-idf (Term Frequency — Inverse Document Frequency)** and use it to obtain a list of ordered results.

Tf-idf is a weighting scheme that assigns each term in a document a weight based on its term frequency (tf) and inverse document frequency (idf).  The higher the scores, more important the term is. 

##### TF
**tf** refers to the frequency of a term $t$ in a specific document $d$. The basic idea is that as a term appears more in the document it becomes more important. On the other side, if we only use pure term counts, longer documents will be favored more. Consider two documents with exactly the same content but one being twice longer by concatenating with itself.  The tf weights of each word in the longer document will be twice the shorter one, although they essentially have the same content. To deal with this issue we need to **normalize the term frequencies**.

$$tf_{t,d} = \dfrac{N_{t,d}}{||D||}\tag{1}$$



where ||D|| is the Euclidean norm. 


Let $D=[t_1, t_2, \dots, t_n]$ be the document vector where $t_i$ represent the frequency of the term $i$, the  Euclidean Norm is calculated as

$$\sqrt{\sum_{t=1}^{n}t_i{^2}}\tag{2}$$

Note that $||D||$ is the same for all terms of a document.


##### IDF
A drawback of tf is that it considers all terms equally important. However, less common terms are more discriminative than others. To deal with this issue we introduce **idf (inverse document frequency)** that takes into account the number of documents containing the term.

$$idf_t = log\dfrac{N}{df_t}\tag{3}$$

where:

- $N$ is the total number of documents;
- $df_t$ is the number of ocuments containing the term $t$.

The log operation is applied to avoid that terms that appears in a high number of documents are considered to be too much less important, in this way we are smoothing (dampening) this difference.


In [16]:
def create_index_tfidf(lines, numDocuments):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numDocuments -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
        
    index=defaultdict(list)
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    titleIndex=defaultdict(str)
    idf=defaultdict(float)
    
    for line in lines.values(): # Remember, lines contain all tweets, each line is a tweet
        #tweetID = line['tweetID']
        tweetID = line['tweetID']        
        terms = line['tokens']
        tweetIndex[tweetID]=line['text']           
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        termdictPage={}

        for position, term in enumerate(terms): ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][tweetID].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweetID, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)


        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]= len(termdictPage[term])  # increment df for current term
        
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
        # Compute idf following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
            
    return index, tf, df, idf, tweetIndex


In [17]:
start_time = time.time()
numTweets = len(cleanTweets)
index, tf, df, idf, tweetIndex = create_index_tfidf(cleanTweets, numTweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 9.58 seconds


In [18]:
def saveDictToCSV(d, filename):
    w = csv.writer(open(filename, "w"))
    for key, val in d.items():
        w.writerow([key, val])
        
def loadDictFromCSV(filename):
    tmpDict={}
    with open(filename,'r') as data: 
        for line in csv.reader(data): 
            tmpDict[line[0]]=line[1]
    return tmpDict

In [27]:
def rankDocuments(terms, docs, index, idf, tf, titleIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    titleIndex -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
        
    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaing elements would became 0 when multiplied to the queryVector
    docVectors=defaultdict(lambda: [0]*len(terms)) # I call docVectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary 
    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for queryVector
    
    query_norm = la.norm(list(query_terms_count.values()))
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]  # TODO: check if multiply for idf

    # calculate the score of each tweet
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine siilarity
    # see np.dot
    
    docScores=[ [np.dot(curDocVec, queryVector)/(np.linalg.norm(curDocVec)*np.linalg.norm(queryVector)), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[(x[0], x[1]) for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ titleIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

In [28]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=cleanTweet(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs = docs.union(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, tweetIndex)   
    return ranked_docs

In [43]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20

print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
for d_score, d_id in ranked_docs[:top] :
    print("Tweet ID= {}\nTweet: {}\nScore: {}\n".format(d_id, tweetIndex[d_id], round(d_score, 4)))

Insert your query:

can I see elections on netflix?

Top 20 results out of 91 for the seached query:

Tweet ID= 1333069486484840450
Tweet: Establishment GOP is obtuse: Trump voters see election fraud as part of rolling coup targeting POTUS. If vote is re… https://t.co/XdvbVpFhOy
Score: 0.8165

Tweet ID= 1333026911799439361
Tweet: Turn off Netflix, NFL, and Fox News  tell you Joe Biden is President elect!!!  You ask what can we do ?  Read… https://t.co/hB9NwTZyy3
Score: 0.8165

Tweet ID= 1333092500102377472
Tweet: The results of the Presidential election do not fit with historic norms, it's highly doubtful that Biden really won… https://t.co/4YOcoakHcf
Score: 0.5774

Tweet ID= 1333092493324398598
Tweet: @thehill Every news outlet that is not Fox/OANN/Newsmax, every election board in states Biden won, every judge find… https://t.co/hvX0iQysQ5
Score: 0.5774

Tweet ID= 1333092481081208833
Tweet: ‘My mind will not change in 6 months’: Trump vows to use ‘125% of my energy’ on hopeless electi